# OCDS Advanced Track Tutorial

## Tutorial Learning Goals

Welcome to the Open Climate Data Science Workshop advanced track tutorial. We are  [Nick Gawron](https://www.linkedin.com/in/ngawrondata/) and [Livia Popa](https://www.linkedin.com/in/livia-popa-23a018183/), and we will be working with you through today's tutorial.  We will be using R studio for today's session. 

We will be tackling these objectives:

- *Define* open data and reproducible science
- *Recognize* the importance of statistical analysis for climate data analysis using cloud based data
- *Apply* techniques to access and explore publicly available climate data from the cloud using exploratory data analysis (i.e. tidyverse)
- *Create* statistical model to predicts results for a representative case study

### Meet Suzie!

Mr. Wuf works for Mount Mitchell State Park in Burnsville, NC and was recently asked by his boss to write a report summarizing rainfall and temperature data for 2021. This report will be used to help optimize 2022 event planning (e.g., fall color viewing) for park visitors and maintenance scheduling for park staff. He recently got promoted to the head office of the NC State Climate office! Mr. Wuf’s wife, Mrs. Wuf, recently told him about the State Climate Office of North Carolina’s new nClimgrid and Econet data portals. He agrees with her that it would be a great opportunity to check out these new, free tools. After some preliminary sleuthing around nClimgrid, he discovered there was a Monthly U.S Climate Gridded Dataset (nClimGrid; https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ncdc:C00332). How did he miss this? Once he downloads these data from nClimGrid and Econet, Mr. Wuf plans to put the skills he learned in an online R programming course to the test for this real-world, work-related project.\


![Suzie](images/suzie.png)

## Importing Data

### Cloud Based Data Introduction 

- Cloud storage is a service model in which data is transmitted and stored on remote storage systems, where it is maintained, managed, backed up and made available to users over a network

- Cloud based data is stored in logical pools across storage servers located on premises or in a data center managed by a third party cloud provider such as amazon web services (aws)


![](images/cloud_data.jpg)

### nClimGrid Data Set

- The NOAA Daily U.S. Climate Gridded Dataset (nClimGrid) consists of four climate variables derived from the station observation networks: maximum temperature, minimum temperature, average temperature and precipitation. 

- Each file provides daily values in a 5 x 5 kilometers grid for the Continental United States. Data is available from 1950s to the present. 

- On an annual basis, approximately one year of "final" nClimGrid will be submitted to replace the initially supplied "preliminary" data for the same time period. 

- We can find nClimGrid Data hosted on Amazon Web Service (AWS) [here](https://noaa-nclimgrid-daily-pds.s3.amazonaws.com/index.html#EpiNOAA/). There are different versions of the data including orginal 5km x 5km grids, county aggregated, and state aggregated data.


In [ ]:
## Set up the environment by loading libraries
#package for plotting 
library(ggplot2)
#package for cleaning data and logical things 
library(tidyverse)

#time series related packages
library(forecast)
library(xts)
library(TSstudio)

#other packages for kmeans
library(here)
library(tidymodels)
library(ClusterR)
library(cluster)
library(broom)

## allows us to read in large amounts of data
library(data.table)

## plotting in 3d
library(plotly)

## library 
library(usmap)

## for date manipulation 
library(lubridate)

## for PCA
library(corrplot)

In [ ]:
## Read in data from AWS nClimGrid 2020-2021
## From the decadal section with state labels
## Drop missing values
nclim_2020s<-read_csv('https://noaa-nclimgrid-daily-pds.s3.amazonaws.com/EpiNOAA/decadal/2020-2021-ste-decadal.csv') %>% 
    drop_na()

In [ ]:
## Read in nClimGrid 2010-2019
## From decadal with state labels
## Use the fread command from data.table
nclim_2010s<-fread('https://noaa-nclimgrid-daily-pds.s3.amazonaws.com/EpiNOAA/decadal/2010-2019-ste-decadal.csv') %>% 
    drop_na()


In [ ]:
#If computation time takes a while ... we have   a .Rda file 
load("NCLIM2010.Rda")
load("NCLIM2020.Rda")

## Functions/Cleaning the Data

### Combining Data Sets from Different Decades

- We are going to make two datasets `nclim` as well as `nclimk_raw`
- `nclim` will be used for time series analyses and `nclimk_raw` will be used for k means clustering

In [ ]:
## Lets quickly combine the data sets to get a data set ranging from the above
## lets also filter for data record that pertain to North Carolina
nclim <- rbind(nclim_2020s,nclim_2010s) %>%
    filter(state == c("North Carolina", "South Carolina"))

In [ ]:
## We are looking at values from January,March,April,July for all the US here, we will eventually subset to a certain state!
## Read in the data with county labels, call it nclimk_raw

nclimk_raw =data.frame()

for (Month_val in 1:2){
  
  Link = paste0('https://noaa-nclimgrid-daily-pds.s3.amazonaws.com/EpiNOAA/csv/20210', Month_val,'-cty-scaled.csv')
  
  nclimk_raw_temp<-read_csv(Link)  

  nclimk_raw = rbind(nclimk_raw, nclimk_raw_temp)
  
}

### Functions

Using the formula $(C \times \frac{9}{5}) + 32=F$, covert the temperature to Fahrenheit. 


In [ ]:
#create a function for basic syntax 
# convert data value temperature F to C
# Call the function CtoF
CtoF<- function(datcols){
  x<- (9/5)*datcols + 32
  return(x)
}

We now will need to apply this function to the temperature columns. 

In [ ]:
# apply the function  CtoF with tidyverse functions 
# to the columns relating to temperature
colnames(nclim)
nclim[8:10]<- nclim[8:10]%>% lapply(CtoF)
head(nclim)

### Cleaning

In [ ]:
## Apply Tidyverse piping to easily add degree labels to the end of certain columns for temp
## and to add units for precipitation
## Inspect 
colnames(nclim)[8:10]<-colnames(nclim)[8:10]%>%tolower()%>% paste0("_degf")
colnames(nclim)[7]<-colnames(nclim)[7]%>%tolower()%>% paste0("_cm")

head(nclim)

In [ ]:
## Keep the weather data columns as well as the date, drop all other columns 
## Further Create the variable ifRain: a factor to indicate whether it rained on a certain day
## Call this final data set nclimf

nclimf <- nclim %>% select(date,prcp_cm,tavg_degf,tmin_degf,tmax_degf,state)%>%
                    mutate(ifRain= as.factor(as.integer(prcp_cm>0)))%>%
                    select(-state)
                                                 
head(nclimf)

### Cleaning for nclimk

Below we do the same cleaning and data preparation that we did for `nclim`, with the exclusion of the ifNC variable. This is because we will be looking at all of the data.

In [ ]:
# we consider a certain state
state2consider="NC" 

nclimk_last <- nclimk_raw%>%
               filter(state==state2consider)%>%
               select(-c(state,month,region_code,year,day))

#applying function CtoF
nclimk_last[4:6]<- nclimk_last[4:6]%>%lapply(CtoF)

# nclim_k colnames 
colnames(nclimk_last)[4:6] <- colnames(nclimk_last)[4:6]%>%
                            tolower()%>%
                            paste0("_degf")

#colnames for percp.
colnames(nclimk_last)[3] <- colnames(nclimk_last)[3]%>%
                          tolower()%>% 
                          paste0("_cm")

nclimk_last$date<-nclimk_last$date%>%as.Date("%m/%e/%Y")

## check the first few rows of the data
head(nclimk_last)

In [ ]:
## check the last few rows of the data
tail(nclimk_last)

## Explanatory Data Analysis

- We will inspect the data through a couple graphs as well as a simple linear regression

In [ ]:
## scatterplot of avg tempurature vs time, color the points red
## add labels
## save the plot at plotOTemp object
plotOtemp<-ggplot(nclim,aes(x=date,y=tavg_degf))+geom_point(colour="red")+labs(x = "Date", y = "Average Tempurature (F)",title ="Tempurature Over Time")
plotOtemp

## Statistical Modeling

### Simple Regression 

- We will perform a simple regression see if that model fits the data well.


In [ ]:
## Using the plotOtemp object create a linear regression for the same data
plotOtemp + geom_smooth(method="lm", se=TRUE, col = "black") + 
    theme_classic()

### Time Series Analysis

- A time series is a collection of observations of well-defined data items obtained through repeated measurements over time

- Time series analyze a sequence of data points collected over an interval of times

- We will now try to fit a time series model to this data rather than a regression to see if the results improve. We will create a time series forecast with ARIMA (Auto Regressive Integrated Moving Average).

- First we begin by creating an `xts` object (a time series project in R).

In [ ]:
## with the variables date and average temperature, create a time series object. 
## save as temp_ts
temp_ts <- xts(nclimf$tavg_degf, nclim$date)

- Now we will split the data into a training and testing set. Majority of the data will be in the training set.

In [ ]:
# train/validation split - we want to split the data set into two groups with
# 70% of data goes to the training data, split by date
train_date <- nclimf$date[round(nrow(temp_ts) * 0.7)]
train <- temp_ts[index(temp_ts) <= train_date]
test <- temp_ts[index(temp_ts) > train_date]

- Next we plot the time series object we created to take a closer look at the data

In [ ]:
# plot the time series object we called temp_ts
plot(temp_ts)

### Time to build the model!

In [ ]:
# build the time series model
model_ts <- auto.arima(as.numeric(train), trace=TRUE)
model_ts

- Now we will plot the test data

In [ ]:
## plot the test data
plot(test)

- Once the model has been created, we will use that to create a forecasting `xts` object

- Finally we plot the validation (testing) data and layer the forecast on top for our time series model

In [ ]:
## plot validation data with forecast using autoplot
forecast <- autoplot(forecast(model_ts, h=length(test)))
forecast

## K-Means Clustering

### Clustering Using nClimGrid County Aggregated Data 

#### Creating Functions

- We want to create a function that can do a complicated process in as few steps as possible. 

- `calculate_cluster` is the name of the function we are going to create to compute k-means clusters among other information. 

In [ ]:
## Create a function to take in data and a number of k clusters
## We want to scale input data, perform k means, and output a dataframe with assigned clusters and silhouette score

calculate_cluster <- function(data, k) {
  x <- data %>%
    na.omit() %>%
    scale()
  
  df <- kmeans(x, center = k) %>%
    augment(data) %>% # creates column ".cluster" with cluster label
    # makes coloumn called silhouette, uses a cluster function called silhouette
    mutate(silhouette = cluster::silhouette(as.integer(.cluster), dist(x))[, "sil_width"])
    # calculate silhouette score
  return(df)
}

In [ ]:
# Convert nclimlast to data frame, call it nclimk
# Use plot_ly to create a 3D scatterplot

nclimk <- data.frame(nclimk_last)
##plot to the data of average tempurature w.r.t date
plot_ly(x=nclimk$prcp_cm, y=nclimk$tavg_degf, z=nclimk$tmax_degf)


In [ ]:
## data subset nclimk
## we want to only look at the numeric variables
## omit any missing values 
## inspect data
nclimk_tocluster <- nclimk %>%
    select(tavg_degf,prcp_cm,tmax_degf) %>%
    na.omit()
  
head(nclimk_tocluster)

- Now we want to set up a data frame to determine which runs 

- Below we will use the `map` function, more help can be found from this
[link](https://www.rdocumentation.org/packages/purrr/versions/0.2.5/topics/map)

- This works similar to `lapply` but runs C++ in the background

In [ ]:
## map cluster calculations function to range of k values
## make cluster and silhoutte scores for each k
## Inspect data

## It is important to set the seed number to make sure 
## the results are reproducible every time when we rerun the code
set.seed(314)
  temp_cluster_data <- tibble(k = 2:12) %>%
    mutate(kclust = map(k, ~calculate_cluster(nclimk_tocluster, .x))) %>%
    unnest(cols = c(kclust))
  
head(temp_cluster_data)

In [ ]:
str(temp_cluster_data)

In [ ]:
## calculate average silhoutte score (highest for optimal number of k clusters)
## for each k value
## store these values for each value of k as temp_sil_score_data
temp_sil_score_data <- temp_cluster_data %>%
    group_by(k) %>%
    summarize(avg_sil_score = mean(silhouette, na.rm = TRUE))

head(temp_sil_score_data) 

In [ ]:
## find maximum
temp_optimal_sil_score_data <- temp_sil_score_data %>%
    filter(max(avg_sil_score, na.rm = TRUE) == avg_sil_score)
  
## save optimal k
  temp_optimal_k_value <- temp_optimal_sil_score_data$k

  temp_optimal_k_value
## 4

 - Note we could have also used a visual inspection

In [ ]:
## plot the data from temp_sil_score_data and visually inspect to determine the k
## with the largest sil score
## Add a marker point to visually show the max value!
temp_sil_score_data%>%
  ggplot(mapping =aes(x=k,y=avg_sil_score))+
  geom_line()+
  geom_point()+
  scale_x_continuous(breaks= pretty_breaks())+
  geom_point(data=temp_sil_score_data%>%
             filter(max(avg_sil_score)==avg_sil_score),
             pch=22,
             size=5,
             colour="purple")+
  labs(x="Value of k",y="Mean of Silhouette Score",title="Silhouette Score for Value of k")+
  theme_light()


# save plot as image 
ggsave("Silhouette_Plot_teacher.png")

In [ ]:
## save optimal k cluster data as nclimk_optimal_cluster
nclimk_optimal_cluster <- temp_cluster_data %>%
    filter(k == temp_optimal_k_value)

head(nclimk_optimal_cluster)
tail(nclimk_optimal_cluster)

In [ ]:
## Combine nclim_optimal_cluster with appropriate county data as well as date
## Update column names
## Call this ClusterWCounty

ClusterWCounty <- cbind(nclimk_optimal_cluster, nclimk$county, nclimk$date)
colnames(ClusterWCounty)[7:8] = c("county","date")
head(ClusterWCounty)

- We have a function to compute the mode of a data set. I.e. This will help us compute the determine the cluster every county most often belongs too. 

In [ ]:
## function for the mode
getmode <- function(v) {
   uniqv <- unique(v)
   uniqv[which.max(tabulate(match(v, uniqv)))]
}

In [ ]:
## Create a common cluster vector for the new data frame called countyplot
## Change the name of the county to the appropriate fips code

## if i was to compute the mode
countyPlot <- ClusterWCounty %>% group_by(county) %>% 
                summarise(Common_Cluster = getmode(.cluster)) %>%
                mutate(fips = fips(state = state2consider, county = county))%>%
                select(-county)

countyPlot<- data.frame(countyPlot)
head(countyPlot)

In [ ]:
# Using plot_usmap to plot North Carolina counties with their appropriate clusters
plot_usmap( data = countyPlot, 
            values = "Common_Cluster", "counties", 
            include = c(state2consider), 
            color="black")+
            labs(title=paste0(state2consider," Cluster Mapping"))+  
            #can comment out the below line if we use the mode method. 
            #scale_fill_continuous(low = "#FFCC00", high = "#CC0000", name="Common_Cluster", label=scales::comma)+
            theme(legend.position="right") 

- Let us examine this topographical map from [geology.com]()

![NC Map](images/NCelv.PNG)

In [ ]:
#3d plot for all of our clusters, same as before but now apply cluster coloring from ClusterWCounty
plot_ly(x = nclimk$prcp_cm, 
        y = nclimk$tavg_degf, 
        z = nclimk$tmax_degf, 
        color = ClusterWCounty$.cluster)

- Now let us look at the data cluster assignment density for Suzie's analysis:  Pender County.  

In [ ]:
## Filter Data by Pender County
PenderCtDOY <- ClusterWCounty%>%filter(county=="Pender County") %>%
                    select(-c(k,county,silhouette))

head(PenderCtDOY)

In [ ]:
## plot outputs density of doy colored by cluster
ggplot(data = PenderCtDOY) +
  geom_density(aes(x = date, fill = .cluster), alpha = 0.5) + 
  labs(x = "Date", y = "Density", fill = "Cluster ID") +
  theme_classic()

- We see a major influence in Pender County 

In [ ]:
## Observe Pender County Weather Data in Clusters 

PenderCtDOY %>%
      select(-c(date)) %>%
      group_by(.cluster) %>%
      summarise(across(everything(), list(mean)))

## Further we can compare the averages acorss 
## MCdoweel and Pender

ClusterWCounty %>%
      filter(county==c("McDowell County","Pender County"))%>%
      select(-c(k,silhouette,date,.cluster))%>%
      group_by(county)%>%
      summarise(across(everything(), list(mean)))

- We can observe stark differences in the tempurature and rain across these counties 
  - Evident why they are in differing clusters
  
- Knowing these differences will better inform Suize's berry growing friends!

## Bonus (for time) Content!

### PCA from ECOnet Data 

In [ ]:
cardinal <- read_csv("cardinal_data.csv", 
     col_types = list(`Average Air Temperature (F)` = col_number(), 
         `Maximum Air Temperature (F)` = col_number(), 
         `Minimum Air Temperature (F)` = col_number(), 
         `Average Experimental Leaf Wetness (mV)` = col_number(), 
         `Total Precipitation (in)` = col_number(), 
         `Average Relative Humidity (%)` = col_number(), 
         `Average Soil Moisture (m3/m3)` = col_number(), 
         `Average Soil Temperature (F)` = col_number(), 
         `Average Solar Radiation (W/m2)` = col_number(), 
         `Average Station Pressure (mb)` = col_number()))

cardinal<-drop_na(cardinal)
str(cardinal)
cardinal$Date<-as.Date(cardinal$Date, tryFormats= c("%m/%d/%y"))
view(cardinal)

#changes col names
colnames(cardinal)=c("date","AvgT","MaxT","MinT","AvgLw","Tprep","AvgHum","AvgSm","AvgSt","AvgSr","AvgStp")

cardinal$IfRain<- (cardinal$Tprep>0)
cardinal$IfRain<-as.factor(as.integer(cardinal$IfRain))

### Basic Plotting with GGplot 

In [ ]:
ggplot(cardinal, aes(x=date, y=AvgT)) + geom_line() + 
    labs(title="Total Daily Rainfall by Date",
         y="Average Tempurature (F) ", 
         x= "Date")

- EDA is how we can motivate future ML models!

- We can use forecasting to extend this trend!

### PCA to cluster rain variable 


- using cardinal data to observe *if* there is clustering 

- used for future models

- helps us describe higher dimensional data with **less**


Three general steps: 

  1. Remove heavily correlated columns! 
    - Min Temp and Max Temp for a certain day will correlate with one another!
    
  2. Center Data

Observe: 

In [ ]:
library(corrplot)
corrplot(cor(cardinal[,-c(1,12)]))

- Tells us to remove all but one temperature variable

In [ ]:
IfRainVar<- cardinal$IfRain
cardshort <- cardinal%>%select(-c(date,IfRain,Tprep,MinT,MaxT))
head(cardshort)

- We will now actually conduct PCA on our data


In [ ]:
pca_card <- princomp(scale(cardshort, scale=FALSE), cor = FALSE)

In [ ]:
plot(pca_card$scores, pch = 16, col =IfRainVar)

legend("topright",c("No Rain","Rain"), pch=16, col=c("black","red"))

- Here we can look at how good PCA does at describing changes in data 

- We see 2 components describes 96% of the data's variation ! (This is very good)

In [ ]:
summary(pca_card)

In [ ]:
screeplot(pca_card, type = "lines")

### How are the original variables related to the principal components?

- Does not print small values, less impactful to correlation 

In [ ]:
loadings(pca_card)

- The loading are simple correlations between the principal components and the original variables (Pearson’s r).

- Values closest to 1 (positive) or -1 (negative) will represent the strongest relationships, with zero being uncorrelated.

We see in PC 1 that there is a high positive correlation between AvgSr. We see the correlation between solar radiation and the component direction is quite high. So by looking at the second component or the y-axis of our previous plot: we see for the most part, Leaf wetness correlated well with the occurance of rain.   


- Another visual to observe the impact of each variable on the principal component!  

- Not super pretty here

In [ ]:
biplot(pca_card)

## Closing & Resources

- *Define* open data and reproducible science
- *Recognize* the importance of statistical analysis for climate data analysis using cloud based data
- *Apply* techniques to access and explore publicly available climate data from the cloud using exploratory data analysis
- *Create* a statistical model to predicts results for a representative case study


### Resouces

- For a A Hydrologists Guide to Open Science, from the HESS Journal, click [here](https://hess.copernicus.org/articles/26/647/2022/). 

### Survey 

Thank you for attending the beginner track tutorial at the Open Climate Data Science Workshop. Please complete the voluntary feedback survey, linked [here](https://docs.google.com/forms/d/e/1FAIpQLScx3Vri5m2qrSYQGodt-bsFyImg-8P57OKCuuH-UadMVinmXA/viewform). The main purposes of this survey is to (1) determine we met specified teaching goals and (2) improve teaching materials for subsequent tutorial sessions.